## goal

Do the same as sample_2_3 but pick a cell (hopefully) where surface and kpp are nonzero

In [37]:
iz,iy,ix = 0,170,9

In [38]:
# auto-reload edited modules
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, "/home/mmurakami/crios_backups/an_helper_functions")

# run the script into the curre nt kernel (vars/functions become available)
%run -i "/home/mmurakami/crios_backups/an_helper_functions/prep_grid_aste_90.py"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(450, 90)
RAC2d (40500,)
(100,)


In [39]:
# we need to load the miniaste

dirroot = "/scratch3/atnguyen/aste_90x150x60/"
dirgrid = dirroot + "GRID_real8/"
dirgridnb = dirroot + "GRID_noblank/"
runstr= "run_c68v_heffmosm3x_layers_lessmem1_viscAHp5em2_it0000_pk0000000001/"
layers_path = dirroot + runstr
extL = "LAYERS"
dirmask = dirroot + "run_template/input_maskTransport/"
dirbudg = layers_path + "diags/BUDG/"
dirdiags = dirbudg
dirstate = layers_path + "diags/STATE/"
dirlayers = layers_path + "diags/LAYERS/"
dirtrsp = layers_path + "diags/TRSP/"

In [40]:
# try to use rdmds
fileprefix = "/scratch3/atnguyen/aste_90x150x60/"
extBasin='run_template/input_maskTransport/'
filename = fileprefix + extBasin + "GATE_transports_v2_mskBasin.bin"
ind = np.fromfile(filename, dtype=np.int32)  # auto-reads .meta for shape/dtype/order
orig_shape = (ind.shape)

ind2d = ind.reshape(ny,nx)

mymsk = np.full((ny,nx),np.nan)
mymsk[ind2d == 57408.0] = 1

# make this smaller
mymsk[:,27:50] = np.nan
mymsk[:160,12:30] = np.nan
mymsk[160:163,15:30] = np.nan

ind = ind.reshape(ny,nx)
mymsk = np.full((ny,nx),np.nan)
mymsk[ind == 57408.0] = 1

# make this smaller
mymsk[:,27:50] = np.nan
mymsk[:160,12:30] = np.nan
mymsk[160:163,15:30] = np.nan

In [41]:
# load the ADV terms for the Barents Sea for this singular box -- single box will all be contributing the same
# to the same T or S bin, so these will be the same when plotted in T or S space
# but we can show this anyways
tsstr = np.array(["0000000002","0000000003"])
t2 = int(tsstr[1]) # for the offline version

In [42]:
t2 = int(tsstr[1])
# 'diags/state_3d_set1'
# read theta and salt averages from the t2 timestep (average)
file_name = "state_3d_set1"
meta_state_3d_set1 = parsemeta(dirstate + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_state_3d_set1["fldList"])
varnames = np.array(["THETA","SALT"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
THETA,its,meta = rdmds(os.path.join(dirstate, file_name),t2,returnmeta=True,rec=recs[0])
SALT,its,meta = rdmds(os.path.join(dirstate, file_name),t2,returnmeta=True,rec=recs[1])

THETA = THETA.reshape(nz,ny,nx)
SALT = SALT.reshape(nz,ny,nx)

In [43]:
print("cell",THETA[iz,iy,ix])
print("e",THETA[iz,iy,ix+1])
print("w",THETA[iz,iy,ix-1])
print("n",THETA[iz,iy+1,ix])
print("s",THETA[iz,iy-1,ix])

# print((THETA[iz,iy,ix+1]+THETA[iz,iy,ix]/)2)

cell 0.2333756536245346
e 0.09543295204639435
w 0.43223249912261963
n 0.3746747076511383
s 0.056596677750349045


In [44]:
tC = THETA[iz,iy,ix]
tE = (THETA[iz,iy,ix+1]+ tC)/2
tW = (THETA[iz,iy,ix-1] + tC)/2
tN = (THETA[iz,iy+1,ix] + tC)/2
tS = (THETA[iz,iy-1,ix] + tC)/2
tT = (THETA[iz+1,iy,ix] + tC)/2   # top
tB = (THETA[iz-1,iy,ix] + tC)/2  # bottom

print(tC)
print(tE,tW,tN,tS)
print(tT,tB) # great these are also separated

0.2333756536245346
0.16440430283546448 0.3328040763735771 0.30402518063783646 0.14498616568744183
0.23527029901742935 0.1166878268122673


In [45]:
# load the TS bins
# also load the bins again so we have them locally
boundsT = np.round(np.squeeze(rdmds(layers_path + "layers2TH")).ravel(),1)
nT = boundsT.size - 1
boundsS = np.round(np.squeeze(rdmds(layers_path + "layers1SLT")).ravel(),1)
nS = boundsS.size - 1
binwidthT = boundsT[1:] - boundsT[:-1]
binwidthS = boundsS[1:] - boundsS[:-1]
binwidthT1 = 0.5 * (binwidthT[1:] + binwidthT[:-1])  # normalize by these
binwidthS1 = 0.5 * (binwidthS[1:] + binwidthS[:-1])
binmidT = (boundsT[1:] + boundsT[:-1]) /2    # bin to these
binmidS = (boundsS[1:] + boundsS[:-1]) /2
nT = nT
nS = nS
nTm1 = nT-1
nSm1 = nS-1

In [46]:
file_name = "budg3d_hflux_set2"
meta_budg3d_hflux_set2 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_hflux_set2["fldList"])
varnames = np.array(["ADVx_TH","ADVy_TH","UVELMASS","VVELMASS"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
ADVx_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
ADVy_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])
UVELMASS,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[2])
VVELMASS,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[3])


ADV_hconv = calc_UV_conv_mod(nfx, nfy,get_aste_faces(ADVx_TH.reshape(nz, ny, nx), nfx, nfy),get_aste_faces(ADVy_TH.reshape(nz, ny, nx), nfx, nfy))
ADV_hconv = ADV_hconv #* hf   # degC·m^3/s at cell centers (matches: ff.DFh = ff.DFh .* hf)
ADVhT = ADV_hconv

print(ADVhT[iz,iy,ix])

-212.01987546069904


In [47]:
# we need DFh, DFr, ADVr

## do the advective convergence
file_name = "budg3d_hflux_set2"
meta_budg3d_hflux_set2 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_hflux_set2["fldList"])
varnames = np.array(["DFxE_TH","DFyE_TH"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
DFxE_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
DFyE_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])


# now 3d zfluxes
file_name = "budg3d_zflux_set1"
meta_budg3d_zflux_set1 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_zflux_set1["fldList"])
varnames = np.array(["DFrE_TH","DFrI_TH"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
DFrE_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
DFrI_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])
DFrE_TH = DFrE_TH.reshape(nz,ny,nx)
DFrI_TH = DFrI_TH.reshape(nz,ny,nx)

DF_hconv = calc_UV_conv_mod(nfx, nfy,get_aste_faces(DFxE_TH.reshape(nz, ny, nx), nfx, nfy),get_aste_faces(DFyE_TH.reshape(nz, ny, nx), nfx, nfy))
DF_hconv = DF_hconv * hf   # degC·m^3/s at cell centers (matches: ff.DFh = ff.DFh .* hf)
DFhT = DF_hconv

trWtopDF = -(DFrE_TH+DFrI_TH)

DFrT = np.zeros((nz,ny,nx),dtype=float)
DFrT[:-1,:,:] = (trWtopDF[:-1] - trWtopDF[1:])

# reshape the DF terms
trWtopDF = trWtopDF.reshape((nz,ny,nx))
DFxE_TH = DFxE_TH.reshape((nz,ny,nx))
DFyE_TH = DFyE_TH.reshape((nz,ny,nx))

# now 3d zfluxes
file_name = "budg3d_zflux_set1"
meta_budg3d_zflux_set1 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_zflux_set1["fldList"])
varnames = np.array(["ADVr_TH"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
ADVr_TH,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
ADVr_TH = ADVr_TH.reshape(nz,ny,nx)

In [48]:
# get the surface values as an array for this cell

# to get the surface term, we need J/s and convert to degC.m^3/s
file_name = 'budg2d_zflux_set1'
meta_budg2d_zflux_set1 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg2d_zflux_set1["fldList"])
varnames = np.array(["TFLUX","oceQsw","SItflux"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])
TFLUX,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[0])
oceQsw,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[1])
SItflux,its,meta = rdmds(os.path.join(dirdiags, file_name),t2,returnmeta=True,rec=recs[2])
TFLUX = TFLUX.reshape(ny,nx)
oceQsw = oceQsw.reshape(ny,nx)
SItflux = SItflux.reshape(ny,nx)

# we need to create zconv_top and swtop
dd = mygrid['RF'][:-1]
swfrac = 0.62*np.exp(dd/0.6)+(1-0.62)*np.exp(dd/20)
swfrac[dd < -200] = 0
swtop=mk3D_mod(swfrac,np.zeros((nz,ny,nx)))*mk3D_mod(RAC*oceQsw,np.zeros((nz,ny,nx)))   # J/s

# zconvtop_heat is here
zconv_top_heat = TFLUX * RAC     # W/m^2 * m^2 = J/s


def surface_contrib_JT(zconv_top_heat, swtop, rcp, fill_last=0.0):
    """
    zconv_top_heat: (ny, nx)
    swtop:          (nz, ny, nx)
    rcp:            scalar
    fill_last:      value for bottom slice (k = nz-1), usually 0.0 or np.nan
    returns:
      JsurfT:       (nz, ny, nx)  # Sv / PSU
    """
    nz, ny, nx = swtop.shape

    eT = zconv_top_heat.reshape(1, ny, nx)  # (1,ny,nx) for broadcast

    J = np.empty_like(swtop, dtype=float)

    # k = 0: (eT - fT[1]) / rcp / dT / dS * 1e-6
    J[0] = (eT[0] - swtop[1]) / rcp if np.ndim(binwidthT)==0 else \
           (eT[0] - swtop[1]) / rcp

    # 1 .. nz-2: -(fT[k+1]-fT[k]) / rcp / dT / dS * 1e-6
    J[1:nz-1] = -(swtop[2:nz] - swtop[1:nz-1]) / rcp

    # bottom slice (k = nz-1): no k+1; choose your boundary convention
    J[-1] = fill_last
    return J

Ft_surftest = surface_contrib_JT(zconv_top_heat,swtop,myparms['rcp'])    # this is in degC.m^3/s

In [49]:
# we need to get the total tendency and get the value of this at the faces (averaging to faces)
dt = 1800

# load the tend from the get_Jterms and plot this
file_name = 'budg3d_snap_set2'
meta_budg3d_snap_set2 = parsemeta(dirdiags + file_name + "." + tsstr[0] + ".meta")
fldlist = np.array(meta_budg3d_snap_set2["fldList"])
varnames = np.array(["THETADR"])
recs = np.array([])
for var in varnames:
    irec = np.where(fldlist == var)
    recs = np.append(recs, irec[0][0])

THETADR = np.full((len(tsstr),nz,ny,nx),np.nan)
for i in range(len(tsstr)):
    thisTHETADR,its,meta = rdmds(os.path.join(dirdiags, file_name),int(tsstr[i]),returnmeta=True,rec=recs[0])
    thisTHETADR = thisTHETADR.reshape(nz,ny,nx)
    THETADR[i] = thisTHETADR

# try to just print the theta at these two snaps
DRF3d = np.tile(mygrid['DRF'][:,np.newaxis,np.newaxis],(1,ny,nx))
vol3d = np.tile(mygrid['DRF'][:,np.newaxis,np.newaxis],(1,ny,nx)) * np.tile(mygrid['RAC'][np.newaxis,:,:],(nz,1,1))

# make samples again - print
print("cell center dt")
print("t at t1:",(THETADR[0]/DRF3d)[iz,iy,ix])
print("t at t2:",(THETADR[1]/DRF3d)[iz,iy,ix])

print("dT = ",(THETADR[1]/DRF3d)[iz,iy,ix] - (THETADR[0]/DRF3d)[iz,iy,ix],)
print("binned at,",(THETADR[1]/DRF3d)[iz,iy,ix])

if (THETADR[1]/DRF3d)[iz,iy,ix] - (THETADR[0]/DRF3d)[iz,iy,ix] >0:
    print("This cell is warming")
else:
    print("This cell is cooling")

print()
# try to figure out why
print(THETADR[0, iz, iy,ix]*mk3D_mod(RAC,THETADR[0])[iz,iy,ix]/dt)
print(THETADR[1, iz, iy,ix]*mk3D_mod(RAC,THETADR[1])[iz,iy,ix]/dt)

dTHETADRdt =  (THETADR[1, :, :,:] - THETADR[0, :,:, :]) / dt    # degC.m/
AB_gT = 0
tmptend=(dTHETADRdt-AB_gT)*mk3D_mod(RAC,dTHETADRdt)   # degC.m/s * m^2 = degC.m^3/s
tmptend = tmptend                          # degC.m^3/s
tmptend[iz,iy,ix]  # this is the value in the original layers

cell center dt
t at t1: 0.23311078246846545
t at t2: 0.2339935648352307
dT =  0.0008827823667652557
binned at, 0.2339935648352307
This cell is warming

2349412.0874624182
2358309.228731967


8897.141269548589

In [50]:
ADVx_TH = ADVx_TH.reshape((nz,ny,nx))
ADVy_TH = ADVy_TH.reshape((nz,ny,nx))

In [51]:
# check that these are binning at the right values
allT = np.array([tE,tW,tN,tS,tT,tB])
alliT = np.zeros((6))

m = 0
for myT in allT:
    for i in range(nT-1):
        # print(tC)
        if myT >= binmidT[i]:
            if myT < binmidT[i+1]:
                iT = i
                print(iT)
                alliT[m] = iT

    m += 1

7
7
7
7
7
7


In [15]:
M_N = np.full((nTm1),0.0)
M_W = np.full((nTm1),0.0)
M_E = np.full((nTm1),0.0)
M_S = np.full((nTm1),0.0)
M_T = np.full((nTm1),0.0)
M_B = np.full((nTm1),0.0)

# also define for the DFh and DFr terms
DFh = np.full((nTm1),0.0)
DFr = np.full((nTm1),0.0)

In [17]:
alliT = alliT.astype(int)
alliT

array([8, 9, 8, 9, 8, 8])

In [19]:
tC

0.7667946815490723

In [21]:
# get and print out the values at the correct indices

print(ADVhT[iz,iy,ix])
print(DF

7093.860418017117
